In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
#import requests
#from bs4 import BeautifulSoup
from datetime import date, datetime, timedelta
#Con esto podemos echar un vistazo a código HTML directamente en la Notebook
from IPython.display import HTML, display

## First I will deal with 2016

In [3]:
file = 'final_2016.csv'
data = pd.read_csv(file)
data.head()

,PatientId,AppointmentID,Gender,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,...,Sea Level Pressure low in,Visibility high ml,Visibility avg ml,Visibility low ml,Wind high mph,Wind avg mph,Wind low mph,Precip sum,event,date
0,2.987250e+13,5642903,F,62,JARDIM DA PENHA,0,1,0,0,0,...,29.95,12,7,6,16,11,-,0.01,Nothing,2016-04-29
1,5.589978e+14,5642503,M,56,JARDIM DA PENHA,0,0,0,0,0,...,29.95,12,7,6,16,11,-,0.01,Nothing,2016-04-29
2,4.262962e+12,5642549,F,62,MATA DA PRAIA,0,0,0,0,0,...,29.95,12,7,6,16,11,-,0.01,Nothing,2016-04-29
3,8.679512e+11,5642828,F,8,PONTAL DE CAMBURI,0,0,0,0,0,...,29.95,12,7,6,16,11,-,0.01,Nothing,2016-04-29
4,8.841186e+12,5642494,F,56,JARDIM DA PENHA,0,1,1,0,0,...,29.95,12,7,6,16,11,-,0.01,Nothing,2016-04-29


In [4]:
print(data['DateAppointment'].min(), data['DateAppointment'].max())
print(data['DateScheduled'].min(), data['DateScheduled'].max())
print(data.shape)
print(data.isnull().sum().sum())

2016-04-29 2016-06-08
2015-11-10 2016-06-08
(110527, 38)
0


In [26]:
fitxer = 'final_201415.csv'
df = pd.read_csv(fitxer)
df.head()

,Age,Gender,DayOfTheWeek,Status,Diabetes,Alcoolism,HiperTension,Handcap,Smokes,Scholarship,...,Sea Level Pressure low in,Visibility high ml,Visibility avg ml,Visibility low ml,Wind high mph,Wind avg mph,Wind low mph,Precip sum,event,date
0,19,M,Wednesday,Show-Up,0,0,0,0,0,0,...,29.95,12,8,6,22,11,-,0.00,Nothing,2015-01-14
1,24,F,Wednesday,Show-Up,0,0,0,0,0,0,...,29.95,12,8,6,16,8,-,0.00,Nothing,2015-08-19
2,4,F,Tuesday,Show-Up,0,0,0,0,0,0,...,29.89,12,6,3,12,5,-,0.39,Thunderstorm,2014-02-18
3,5,M,Thursday,Show-Up,0,0,0,0,0,0,...,30.18,12,8,6,12,5,-,0.00,Nothing,2014-08-07
4,38,M,Tuesday,Show-Up,0,0,0,0,0,0,...,29.92,12,7,6,18,9,-,0.00,Rain,2015-10-27


In [4]:
df.columns

Index(['Age', 'Gender', 'DayOfTheWeek', 'Status', 'Diabetes', 'Alcoolism',
       'HiperTension', 'Handcap', 'Smokes', 'Scholarship', 'Tuberculosis',
       'Sms_Reminder', 'AwaitingTime', 'DateAppointment', 'TimeAppointment',
       'DateScheduled', 'TimeScheduled', 'dia', 'temp_high_f', 'temp_avg_f',
       'Temp low F', 'Dew Point high F', 'Dew_Point_avg_F', 'Dew Point low F',
       'humidity high %', 'humidity avg %', 'humidity low %',
       'Sea Level Pressure high in', 'Sea Level Pressure avg (in)',
       'Sea Level Pressure low in', 'Visibility high ml', 'Visibility avg ml',
       'Visibility low ml', 'Wind high mph', 'Wind avg mph', 'Wind low mph',
       'Precip sum', 'event', 'date'],
      dtype='object')

### Next Steps:
   [1. Reducing number of columns.](#Reducing columns)
   
   [2. Creating new feature: dayofweek (where necessary).](#DayOfWeek)

   [3. Days in between appointment and schedule (where necessary).](#awaitingTime)

   [4. Repeat patient (where possible).](#Repeat)
   
   [5. Mapping (where necessary).](#mapping)

  -[6. More cleaning and maybe Time of day (if possible).](#more)

<a id='Reducing columns'></a>
## 1. Reducing columns.

In [5]:
useless_columns = ['dia', 'temp_high_f', 'Temp low F', 'Dew Point high F',
       'Dew Point low F', 'humidity high %','humidity low %', 'Sea Level Pressure high in',
       'Sea Level Pressure avg (in)', 'Sea Level Pressure low in',
       'Visibility high ml', 'Visibility low ml',
       'Wind high mph', 'Wind low mph', 'date']

In [7]:
#final_columns = set(data.columns) - set(useless_columns)
final_columns = [c for c in data.columns if c not in useless_columns]
#final_columns = list(final_columns)

print(len(data.columns)-(len(useless_columns)))
print(len(final_columns))

In [9]:
new_data = data[final_columns]
new_data.head()

,PatientId,AppointmentID,Gender,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,...,TimeScheduled,DateAppointment,TimeAppointment,temp_avg_f,Dew_Point_avg_F,humidity avg %,Visibility avg ml,Wind avg mph,Precip sum,event
0,2.987250e+13,5642903,F,62,JARDIM DA PENHA,0,1,0,0,0,...,18:38:08,2016-04-29,00:00:00,73,66,76,7,11,0.01,Nothing
1,5.589978e+14,5642503,M,56,JARDIM DA PENHA,0,0,0,0,0,...,16:08:27,2016-04-29,00:00:00,73,66,76,7,11,0.01,Nothing
2,4.262962e+12,5642549,F,62,MATA DA PRAIA,0,0,0,0,0,...,16:19:04,2016-04-29,00:00:00,73,66,76,7,11,0.01,Nothing
3,8.679512e+11,5642828,F,8,PONTAL DE CAMBURI,0,0,0,0,0,...,17:29:31,2016-04-29,00:00:00,73,66,76,7,11,0.01,Nothing
4,8.841186e+12,5642494,F,56,JARDIM DA PENHA,0,1,1,0,0,...,16:07:23,2016-04-29,00:00:00,73,66,76,7,11,0.01,Nothing


In [10]:
print(new_data['DateAppointment'].min(), new_data['DateAppointment'].max())
print(new_data['DateScheduled'].min(), new_data['DateScheduled'].max())

2016-04-29 2016-06-08
2015-11-10 2016-06-08


In [11]:
new_data.isnull().sum().sum()

0

<a id='DayOfWeek'></a>
## 2. Day of the Week New Feature

In [12]:
##Day_of_week
new_data['day_of_week'] = pd.to_datetime(new_data['DateAppointment']).dt.weekday_name
new_data['day_of_week'].head()

/Users/fcbnyc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


0    Friday
1    Friday
2    Friday
3    Friday
4    Friday
Name: day_of_week, dtype: object

<a id='awaitingTime'></a>
## 3. AwaitingTime New Feature

In [14]:
new_data['awaiting_time'] =pd.to_datetime(new_data['DateScheduled']) - pd.to_datetime(new_data['DateAppointment'])
new_data['awaiting_time'] = new_data['awaiting_time'].dt.days

/Users/fcbnyc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/fcbnyc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [15]:
new_data['awaiting_time'].unique()

array([   0,   -2,   -3,   -1,   -4,   -9,  -29,  -10,  -23,  -11,  -18,
        -17,  -14,  -28,  -24,  -21,  -15,  -16,  -22,  -43,  -30,  -31,
        -42,  -32,  -56,  -45,  -46,  -39,  -37,  -38,  -44,  -50,  -60,
        -52,  -53,  -65,  -67,  -91,  -66,  -84,  -78,  -87, -115, -109,
        -63,  -70,  -72,  -57,  -58,  -51,  -59,  -41,  -49,  -73,  -64,
        -20,  -33,  -34,   -6,  -35,  -36,  -12,  -13,  -40,  -47,   -8,
         -5,   -7,  -25,  -26,  -48,  -27,  -19,  -61,  -55,  -62, -176,
        -54,  -77,  -69,  -83,  -76,  -89,  -81, -103,  -79,  -68,  -75,
        -85, -112,    1,  -80,  -86,  -98,  -94, -142, -155, -162, -169,
       -104, -133, -125,  -96,  -88,  -90, -151, -126, -127, -111, -119,
        -74,  -71,  -82, -108, -110, -102, -122, -101, -105,  -92,  -97,
        -93, -107,  -95,    6, -139, -132, -179, -117, -146, -123])

<a id='Repeat'></a>
## 4. Repeat Patient

In [16]:
new_data['PatientId'] = new_data["PatientId"].astype(int)

#data['PatientId'].value_counts().index.tolist()
new_data['duplicated'] = new_data.duplicated(['PatientId'])##I want all to be marked as duplicate
new_data['duplicated'].head()

new_data['patient_repeat'] = new_data['duplicated'].astype(int)
print(new_data['patient_repeat'].value_counts())

0    62299
1    48228
Name: patient_repeat, dtype: int64


/Users/fcbnyc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/fcbnyc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/fcbnyc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

<a id='mapping'></a>
## 5. Mapping

In [17]:
### target feature
new_data['No-show'] = new_data['No-show'].map({'Yes': 1, 'No': 0})
###Gender Feature
new_data['Gender'] = new_data['Gender'].map({'F': 1, 'M': 0})

/Users/fcbnyc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/fcbnyc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [18]:
new_data['event'] = new_data['event'].map({'Nothing': 0, 'Fog': 1, 'Rain': 2, 'Thunderstorm': 3})
new_data['event'].unique()

/Users/fcbnyc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


array([0, 1, 3, 2])

In [62]:
### target feature
df['Status'] = df['Status'].map({'No-Show': 1, 'Show-Up': 0})
###Gender Feature
df['Gender'] = df['Gender'].map({'F': 1, 'M': 0})

In [64]:
new_data.shape

(110527, 27)

<a id='more'></a>
## 6. More cleaning.

In [50]:
new_data['TimeScheduled'] = pd.to_datetime(new_data['TimeScheduled'])
new_data = new_data.assign(session=pd.cut(new_data.TimeScheduled.dt.hour,\
                               [0,6,12,18,24],labels=[0,1,2,3]))

/Users/fcbnyc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [54]:
new_data.shape

(110527, 28)

In [70]:
new_data['session'] = new_data['session'].astype(float)

In [52]:
new_data[['TimeScheduled', 'session']].head()

,TimeScheduled,session
0,2017-11-29 18:38:08,2
1,2017-11-29 16:08:27,2
2,2017-11-29 16:19:04,2
3,2017-11-29 17:29:31,2
4,2017-11-29 16:07:23,2


# Dropping unnecessary columns: duplicated and Time Appointment.
        ## The first one is replace by Patient Repeat
        ## The second one has no use if is only 00.

In [59]:
new_data.drop(['duplicated', 'TimeAppointment'], axis=1, inplace=True)

In [60]:
print(new_data.shape)

(110527, 26)


In [71]:
new_data.dtypes

PatientId                     int64
AppointmentID                 int64
Gender                        int64
Age                           int64
Neighbourhood                object
Scholarship                   int64
Hipertension                  int64
Diabetes                      int64
Alcoholism                    int64
Handcap                       int64
SMS_received                  int64
No-show                       int64
DateScheduled                object
TimeScheduled        datetime64[ns]
DateAppointment              object
temp_avg_f                    int64
Dew_Point_avg_F               int64
humidity avg %                int64
Visibility avg ml             int64
Wind avg mph                  int64
Precip sum                  float64
event                         int64
day_of_week                  object
awaiting_time                 int64
patient_repeat                int64
session                     float64
dtype: object

## Separating columns by type

# Dummies: day of week, neighborhood, ...

In [72]:
rars = new_data.select_dtypes(exclude=['object', 'float', 'integer'])
print(rars.columns, len(rars.columns))
objectes = new_data.select_dtypes(include=['object'])
print(objectes.columns, len(objectes.columns))
numeric = new_data.select_dtypes(include=['float', 'integer'])
print(numeric.columns, len(numeric.columns))

Index(['TimeScheduled'], dtype='object') 1
Index(['Neighbourhood', 'DateScheduled', 'DateAppointment', 'day_of_week'], dtype='object') 4
Index(['PatientId', 'AppointmentID', 'Gender', 'Age', 'Scholarship',
       'Hipertension', 'Diabetes', 'Alcoholism', 'Handcap', 'SMS_received',
       'No-show', 'temp_avg_f', 'Dew_Point_avg_F', 'humidity avg %',
       'Visibility avg ml', 'Wind avg mph', 'Precip sum', 'event',
       'awaiting_time', 'patient_repeat', 'session'],
      dtype='object') 21


In [78]:
renew_data.shape

(110527, 117)

In [77]:
renew_data = pd.concat([new_data, pd.get_dummies(new_data['day_of_week'], prefix='dow'),\
                  pd.get_dummies(new_data['Neighbourhood'], prefix='hood'), pd.get_dummies(new_data['event'], prefix='event')], axis=1)
renew_data

,PatientId,AppointmentID,Gender,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,...,hood_SÃO CRISTÓVÃO,hood_SÃO JOSÉ,hood_SÃO PEDRO,hood_TABUAZEIRO,hood_UNIVERSITÁRIO,hood_VILA RUBIM,event_0,event_1,event_2,event_3
0,29872499824296,5642903,1,62,JARDIM DA PENHA,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,558997776694438,5642503,0,56,JARDIM DA PENHA,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,4262962299951,5642549,1,62,MATA DA PRAIA,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,867951213174,5642828,1,8,PONTAL DE CAMBURI,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,8841186448183,5642494,1,56,JARDIM DA PENHA,0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,0
5,95985133231274,5626772,1,76,REPÚBLICA,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
6,733688164476661,5630279,1,23,GOIABEIRAS,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
7,3449833394123,5630575,1,39,GOIABEIRAS,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
8,56394729949972,5638447,1,21,ANDORINHAS,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
9,78124564369297,5629123,1,19,CONQUISTA,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [53]:
d2016 = data[final_columns]
d2016.columns

Index(['GOIABEIRAS', 'GOIABEIRAS', 'SANTA MARTHA', 'SANTA MARTHA',
       'SANTA HELENA', 'SANTA HELENA', 'PONTAL DE CAMBURI',
       'PONTAL DE CAMBURI', 'PatientId', 'Monday',
       ...
       'ARIOVALDO FAVALESSA', 'Tuesday', 'Tuesday', 'Tuesday', 'Rain',
       'FONTE GRANDE', 'FONTE GRANDE', 'COMDUSA', 'COMDUSA', 'humidity avg %'],
      dtype='object', length=211)

In [79]:
renew_data.to_csv('appointments2016.csv',index=False, encoding='utf-8')

# Columns to ignore:
DateApp, AppointmetID, PatientId, Neighborhood, dayof week
# Doublechek all the rest

In [17]:
d201415 = df[final_columns]
d201415.columns

Index(['Smokes', 'Precip sum', 'temp_avg_f', 'Scholarship', 'Wind avg mph',
       'Status', 'Age', 'DateScheduled', 'DateAppointment', 'Tuberculosis',
       'Alcoolism', 'DayOfTheWeek', 'Diabetes', 'event', 'TimeAppointment',
       'HiperTension', 'Dew_Point_avg_F', 'TimeScheduled', 'Gender',
       'Sms_Reminder', 'Handcap', 'AwaitingTime', 'humidity avg %',
       'Visibility avg ml'],
      dtype='object')

Setting a model and the baseline for 2016